In [ ]:
!pip install rdkit-pypi

In [ ]:
!pip install numpy==1.25.0

In [ ]:
import pandas as pd
import random
import os
import numpy as np
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
import math
import os
import torch
import random


from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error

import itertools
import math
import warnings
warnings.filterwarnings(action='ignore')

train = pd.read_csv('train_final.csv')
test = pd.read_csv('test_final.csv')
test.drop(['ID'],axis=1,inplace=True)
submission = pd.read_csv('sample_submission.csv')
train

# TTA

In [ ]:
from rdkit import Chem
import pandas as pd
import random

SEED = 724
random.seed(SEED)

n_augments = 30
augmented_data = []

for idx, row in test.iterrows():
    original_smiles = row['Canonical_Smiles']
    mol = Chem.MolFromSmiles(original_smiles)
    if mol is None:
        continue

    seen = set()
    attempts = 0
    max_attempts = 500

    while len(seen) < n_augments and attempts < max_attempts:
        atom_indices = list(range(mol.GetNumAtoms()))
        random.Random(SEED + idx * 1000 + attempts).shuffle(atom_indices)
        new_order = Chem.RenumberAtoms(mol, atom_indices)
        rand_smiles = Chem.MolToSmiles(new_order, canonical=False)
        if rand_smiles != original_smiles:
            seen.add(rand_smiles)
        attempts += 1

    for smi in seen:
        new_row = row.to_dict()
        new_row['Canonical_Smiles'] = smi
        new_row['MolID'] = idx
        augmented_data.append(new_row)

augmented_test_df = pd.DataFrame(augmented_data)
augmented_test_df.to_csv(f'test_final_aug_{SEED}.csv')
augmented_test_df